In [1]:
import warnings
import pandas as pd
import numpy as np
import json
import ast
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter

pd.set_option('display.max_columns', None)

In [2]:
# Fungsi untuk membersihkan komentar
def clean_comment(comment):
    # Menghapus karakter khusus, emotikon, dan simbol lainnya
    cleaned_comment = re.sub(r'[^\w\s]', '', comment)
    # Tokenisasi kata
    tokens = word_tokenize(cleaned_comment)
    # Menghapus stopwords seperti "the", "and", "is", dll.
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    # Lematisasi kata ke bentuk dasarnya
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    # Menggabungkan kembali token menjadi kalimat
    cleaned_comment = ' '.join(lemmatized_tokens)
    return cleaned_comment

# Fungsi untuk menganalisis komentar
def analyze_comments(data):
    # Membersihkan komentar
    data['cleaned_comment'] = data['comment'].apply(clean_comment)
    # Menghitung frekuensi kata-kata
    word_freq = Counter(" ".join(data['cleaned_comment']).split()).most_common(10)
    # Mencetak kata-kata yang paling sering muncul
    print("Top 10 Most Common Words:")
    for word, freq in word_freq:
        print(f"{word}: {freq}")

In [3]:
df_jurnal = pd.read_csv('jurnals_april.csv')

In [4]:
id_jurnal = 104520
df = df_jurnal[df_jurnal['id'] == id_jurnal]

In [34]:
dictionary = ast.literal_eval(df['userliked'].unique()[0])
df_userliked = pd.DataFrame.from_dict(dictionary, orient='index', columns=['user_id_liked'])
df_userliked['id'] = id_jurnal

In [35]:
df = df.merge(df_userliked, 'left', on='id')

In [58]:
df_comments = pd.read_csv(f'jurnal_comments_{id_jurnal}.csv')

In [59]:
# Clean text from emoticon
df_comments['comment_cleaned'] = [re.sub(r'[^\w\s]', '', x).lower() for x in df_comments['comment']]

In [60]:
# Drop duplicated comment from users
df_comments = df_comments.drop_duplicates(subset=['jurnal_id', 'comment_cleaned'], keep='first')
df_comments = df_comments[df_comments['comment_cleaned'] != '']



In [61]:
# tokenize text
df_comments['comment_token'] = df_comments['comment_cleaned'].str.split()

In [62]:
from wordcloud  import WordCloud

In [64]:
comments = df_comments['comment_cleaned'].tolist()


In [65]:
("").join(comments)

'nice infoterimakasihttwrimakasierimakasihthanks good articlelove alhamdulillah bisa sarapan lagi tanggal 10nicemakasih umaminmakasihmantaplebaran memang identik dengan makanan enak terutama makanan bersantanair putih nih yang utamamantap terima kasihminum mineral aja ya umaminmksh tipsnya minterimakasih tipsnyaterima kasih banyak tipsnya umaminsukses selalusiap umaminmantaaapmakasih infonya minsayniatku mau diet selama bulan ramadhan gagal mulu wkwkwkwkselamat lebaranterimakasih umaminthanks tipsnyasangat bermanfaat tipsnyamksh tipsnya umaminmksh minnice info mininfo yang bermanfaat nice onfopas bgt nih makasih infonya umaminiyes tips makan sehat sangat bermanfaattetap jaga porsi makan saat lebaranthx infonya umaminmakasih tipsnya umaminterimakasih informasinya thank u tipsnyamakasi'

In [7]:
import sqlalchemy as sqla
import sshtunnel as stunnel

In [8]:
mysql_host = "127.0.0.1"
mysql_user = "du_root"
mysql_pwd = "UmamiAjinomoto120114!%40#"
mysql_db = "dapurumami_prod_db"
mysql_port = 3306

ssh_host = "68.183.181.15"
ssh_user = "root"
ssh_pwd = "MT6CHO1FFFHpvbi"
ssh_port = 22

In [9]:
from sqlalchemy import create_engine
from sshtunnel import SSHTunnelForwarder

# Establish SSH tunnel
with SSHTunnelForwarder(
    (ssh_host, ssh_port),
    ssh_username=ssh_user,
    ssh_password=ssh_pwd,
    remote_bind_address=(mysql_host, mysql_port),
) as server:
    # Local port to connect to MySQL through the SSH tunnel
    local_port = server.local_bind_port

    # MySQL connection string
    mysql_conn_str = f"mysql+mysqlconnector://{mysql_user}:{mysql_pwd}@{mysql_host}:{local_port}/{mysql_db}"

    # Create SQLAlchemy engine
    engine = create_engine(mysql_conn_str)

    # Test the connection
    try:
        with engine.connect():
            print("Connection successful!")
            # Perform database operations here
    except Exception as e:
        print("Connection failed:", e)

In [234]:
server.stop()

In [236]:
# SSH Parameters
server = stunnel.SSHTunnelForwarder(
    (ssh_host, ssh_port),
    ssh_username=ssh_user,
    ssh_password=ssh_pwd,
    remote_bind_address=(mysql_host, mysql_port),
)
server.start()

local_port = server.local_bind_port


engine = sqla.create_engine(
    f"mysql+pyodbc://{mysql_user}:UmamiAjinomoto120114!%40#@{mysql_host}:{local_port}/{mysql_db}?driver=MySQL+ODBC+8.3+Unicode+Driver&MULTI_HOST=1",
    connect_args={"TrustServerCertificate": "yes"},
).connect()

DBAPIError: (pyodbc.Error) ('HY000', "[HY000] [MySQL][ODBC 8.3(w) Driver]Access denied for user 'du_root'@'localhost' (using password: YES) (1045) (SQLDriverConnect); [HY000] [MySQL][ODBC 8.3(w) Driver]Access denied for user 'du_root'@'localhost' (using password: YES) (1045)")
(Background on this error at: https://sqlalche.me/e/14/dbapi)

In [219]:
import pyodbc
available_drivers = pyodbc.drivers()
print(available_drivers)


['SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'ODBC Driver 17 for SQL Server', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)', 'PostgreSQL ANSI(x64)', 'PostgreSQL Unicode(x64)', 'PostgreSQL ANSI', 'PostgreSQL Unicode', 'PostgreSQL ODBC Driver(ANSI)', 'PostgreSQL ODBC Driver(UNICODE)', 'MySQL ODBC 8.3 ANSI Driver', 'MySQL ODBC 8.3 Unicode Driver']


In [203]:
offset = 0
query = f"""SELECT * 
FROM points p
WHERE created_at >= '2024-04-01' 
    AND point_source != 'register' 
LIMIT 200 OFFSET {offset}"""

In [204]:
df_user_points = pd.read_sql(query, engine)

InterfaceError: (pyodbc.InterfaceError) ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/14/rvf5)